In [ ]:
#!git clone https://github.com/facebookresearch/VideoPose3D.git
#!cp -ar VideoPose3D/* .
!mkdir "checkpoint"
#!curl https://dl.fbaipublicfiles.com/video-pose-3d/pretrained_h36m_detectron_coco.bin --output "checkpoint/pretrained_h36m_detectron_coco.bin"

mkdir: cannot create directory ‘checkpoint’: File exists


In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-ad5z4lop
  Running command git clone -q https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-ad5z4lop


In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor

from tqdm import tqdm
import subprocess as sp
import numpy as np
import time
import argparse
import sys
import os
import glob
import cv2

In [ ]:
import torch

print(torch.__version__)

1.13.0+cu116


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def gen_skeleton(frame, connections, height, width):
    img_3 = np.zeros([height, width,3],dtype=np.uint8)
    img_3.fill(255)

    # add circles 
    for coord in frame:
        x, y = int(width*coord[0]), int(height*coord[1])
        img_3 = cv2.circle(img_3, center=(x,y), radius=1, color=(255, 0, 0), thickness=6)

    # add lines 
    mapping_list = [[0, 1], [1, 3], [0, 2], [2, 4],
                   [5, 6], [5, 7], [7, 9], [6, 8], [8, 10],
                   [5, 11], [6, 12], [11, 12],
                   [11, 13], [12, 14], [13, 15], [14, 16]]
    for line in mapping_list:
        i, j = line
        st = frame[i, :]
        start_point = (int(width*st[0]), int(height*st[1]))

        en = frame[j, :]
        end_point = (int(width*en[0]), int(height*en[1]))

        img3_ = cv2.line(img_3, start_point, end_point, color=(0, 0, 0), thickness=3)

    return img_3
"""
def gen_skeletons(frame1 , frame2, connections, height, width):
    img_3 = np.zeros([height, width,3],dtype=np.uint8)
    img_3.fill(255)

    # add circles for the frame-set-1
    for coord in frame1:
        x, y = int(width*coord[0]), int(height*coord[1])
        img_3 = cv2.circle(img_3, center=(x,y), radius=1, color=(255, 0, 0), thickness=6)

    # add circles for the frame-set-2
    for coord in frame2:
        x, y = int(width*coord[0]), int(height*coord[1])
        img_3 = cv2.circle(img_3, center=(x,y), radius=1, color=(0, 255, 0), thickness=6)

    # add lines 
    mapping_list = [(0, 1), (1, 3), (3, 5), (0, 2), (2, 4), (0, 6), (1, 7), (6, 7), (6, 8), (7, 9), (8, 10), (9, 11)]
    for line in mapping_list:
        i, j = line

        st = frame1[i, :]
        start_point = (int(width*st[0]), int(height*st[1]))
        en = frame1[j, :]
        end_point = (int(width*en[0]), int(height*en[1]))
        img3_ = cv2.line(img_3, start_point, end_point, color=(250, 0, 0), thickness=3)

        st = frame2[i, :]
        start_point = (int(width*st[0]), int(height*st[1]))
        en = frame2[j, :]
        end_point = (int(width*en[0]), int(height*en[1]))
        img3_ = cv2.line(img_3, start_point, end_point, color=(0, 250, 0), thickness=3)

    return img_3
"""
def gen_video(points, save_file, frame_h, frame_w):
    # make 3D if points are flatten 
    if len(points.shape) == 2:
        fts = points.shape[1]
        x_cds = list(range(0, fts, 3))
        y_cds = list(range(1, fts, 3))
        z_cds = list(range(2, fts, 3))
        points = np.transpose(np.array([points[:, x_cds], points[:, y_cds], points[:, z_cds]]), (1,2,0))

    size = (frame_w, frame_h)
    result = cv2.VideoWriter(save_file, 
                         cv2.VideoWriter_fourcc(*'MJPG'),
                         10, size)

    # mapping_list = [(11, 12), (11, 13), (13, 15), (12, 14), (14, 16), (12, 24), (11, 23), (23, 24), (24, 26), (26, 28), (23, 25), (25, 27)]
    mapping_list = [(0, 1), (1, 3), (3, 5), (0, 2), (2, 4), (0, 6), (1, 7), (6, 7), (6, 8), (7, 9), (8, 10), (9, 11)]
    for __id,frame in enumerate(points):
        skel_image = gen_skeleton(frame, mapping_list, frame_h, frame_w)
        result.write(skel_image)

    result.release()
"""
def gen_cmp_video(points1, points2, save_file, frame_h, frame_w):
    # make 3D if points are flatten 
    if len(points1.shape) == 2:
        fts = points1.shape[1]
        x_cds = list(range(0, fts, 3))
        y_cds = list(range(1, fts, 3))
        z_cds = list(range(2, fts, 3))
        points1 = np.transpose(np.array([points1[:, x_cds], points1[:, y_cds], points1[:, z_cds]]), (1,2,0))

    if len(points2.shape) == 2:
        fts = points2.shape[1]
        x_cds = list(range(0, fts, 3))
        y_cds = list(range(1, fts, 3))
        z_cds = list(range(2, fts, 3))
        points2 = np.transpose(np.array([points2[:, x_cds], points2[:, y_cds], points2[:, z_cds]]), (1,2,0))

    size = (frame_w, frame_h)
    result = cv2.VideoWriter(save_file, 
                         cv2.VideoWriter_fourcc(*'MJPG'),
                         10, size)

    # mapping_list = [(11, 12), (11, 13), (13, 15), (12, 14), (14, 16), (12, 24), (11, 23), (23, 24), (24, 26), (26, 28), (23, 25), (25, 27)]
    mapping_list = [(0, 1), (1, 3), (3, 5), (0, 2), (2, 4), (0, 6), (1, 7), (6, 7), (6, 8), (7, 9), (8, 10), (9, 11)]
    for __id,frame_1 in enumerate(points1):
        frame_2 = points2[__id]
        skel_image = gen_skeletons(frame_1, frame_2, mapping_list, frame_h, frame_w)
        result.write(skel_image)

    result.release()
"""
def convert_array(input):
  arr = input.reshape(-1,input.shape[1]*input.shape[2])
  return arr

In [ ]:
args = {
    "cfg":"COCO-Keypoints/keypoint_rcnn_R_101_FPN_3x.yaml",
    "image_ext":"mp4"
}

data_fol = "/content/drive/MyDrive/22_FYP42 - Zero-shot Explainable HAR/Datasets/knn_handpicked/5_sample"
path_parts = data_fol.split("/Datasets/")[1]

model_ident = "detectron2"
main_npz_directory = f"/content/drive/MyDrive/22_FYP42 - Zero-shot Explainable HAR/Datasets/skeleton_npz_outputs/{model_ident}/{path_parts}"
main_vid_directory = f"/content/drive/MyDrive/22_FYP42 - Zero-shot Explainable HAR/Datasets/skeleton_reconstruction_outputs/{model_ident}/{path_parts}"

In [ ]:
main_npz_directory

'/content/drive/MyDrive/22_FYP42 - Zero-shot Explainable HAR/Datasets/skeleton_npz_outputs/detectron2/knn_handpicked/5_sample'

In [ ]:
convert_array(np.random.randn(177,17,2)).shape

(177, 34)

In [ ]:
gen_video(np.random.randn(177,17,2), "lol.mp4", 400, 400)

In [ ]:
def get_resolution(filename):
    command = ['ffprobe', '-v', 'error', '-select_streams', 'v:0',
               '-show_entries', 'stream=width,height', '-of', 'csv=p=0', filename]
    pipe = sp.Popen(command, stdout=sp.PIPE, bufsize=-1)
    for line in pipe.stdout:
        w, h = line.decode().strip().split(',')
        return int(w), int(h)

def read_video(filename):
    w, h = get_resolution(filename)

    command = ['ffmpeg',
            '-i', filename,
            '-f', 'image2pipe',
            '-pix_fmt', 'bgr24',
            '-vsync', '0',
            '-vcodec', 'rawvideo', '-']

    pipe = sp.Popen(command, stdout=sp.PIPE, bufsize=-1)
    while True:
        data = pipe.stdout.read(w*h*3)
        if not data:
            break
        yield np.frombuffer(data, dtype='uint8').reshape((h, w, 3))


def main(args):

    cfg = get_cfg()
    cfg.MODEL.DEVICE = "cpu"
    cfg.merge_from_file(model_zoo.get_config_file(args["cfg"]))
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(args["cfg"])
    predictor = DefaultPredictor(cfg)
    

    if os.path.isdir(args["im_or_folder"]):
        im_list = glob.iglob(args["im_or_folder"] + '/*.' + args["image_ext"])
    else:
        im_list = [args["im_or_folder"]]

    for video_name in im_list:
        out_name = os.path.join(
                args["output_dir"], os.path.basename(video_name)
            )
        
        vidout_name = os.path.join(
                args["video_out"], os.path.basename(video_name)
            )

        boxes = []
        segments = []
        keypoints = []

        t = time.time()
        for frame_i, im in enumerate(read_video(video_name)):
            outputs = predictor(im)['instances'].to('cpu')
            
            #print('Frame {} processed in {:.3f}s'.format(frame_i, time.time() - t))

            has_bbox = False
            if outputs.has('pred_boxes'):
                bbox_tensor = outputs.pred_boxes.tensor.numpy()
                if len(bbox_tensor) > 0:
                    #print(outputs.scores.numpy())
                    has_bbox = True
                    #scores = outputs.scores.numpy()[:, None]
                    bbox_tensor = bbox_tensor[0]# np.concatenate((bbox_tensor, scores), axis=1)
            if has_bbox:
                kps = outputs.pred_keypoints.numpy()
                kps_xy = kps[:, :, :2]
                kps = kps_xy[0]
                #kps_prob = kps[:, :, 2:3]
                #kps_logit = np.zeros_like(kps_prob) # Dummy
                #kps = np.concatenate((kps_xy, kps_logit, kps_prob), axis=2)
                #kps = kps.transpose(0, 2, 1)
            else:
                kps = []
                bbox_tensor = []
                
            # Mimic Detectron1 format
            #cls_boxes = [[], bbox_tensor]
            #cls_keyps = [[], kps]
            
            boxes.append(bbox_tensor)
            #segments.append(None)
            keypoints.append(kps)

        
        # Video resolution
        metadata = {
            'w': im.shape[1],
            'h': im.shape[0],
        }
        try:
        
          keypoints = np.array(keypoints)

          keypoints[:,:,0] = keypoints[:,:,0]/metadata["w"]
          keypoints[:,:,1] = keypoints[:,:,1]/metadata["h"]

          #print(keypoints,metadata)
          gen_video(keypoints, vidout_name, int(metadata["h"]*0.25), int(metadata["w"]*0.25))
          print('Video {} processed in {:.3f}s shape {}'.format(video_name, time.time() - t,keypoints.shape))
          np.savez_compressed(out_name, boxes=boxes, keypoints=keypoints, metadata=metadata)
        except Exception as exp:
          print(exp)

In [ ]:
for cls in tqdm(os.listdir(data_fol)):
  cls_fol = os.path.join(data_fol,cls)
  out_fol = os.path.join(main_npz_directory,cls)
  vid_fol = os.path.join(main_vid_directory,cls)

  os.makedirs(out_fol,exist_ok =True)
  os.makedirs(vid_fol,exist_ok =True)
  args["im_or_folder"] = cls_fol
  args["output_dir"] = out_fol
  args["video_out"] = vid_fol
  print(f"Processing class :- {cls}")
  main(args)

  0%|          | 0/15 [00:00<?, ?it/s]

Processing class :- watching_TV



model_final_997cc7.pkl: 0.00B [00:00, ?B/s]
model_final_997cc7.pkl:   0%|          | 8.19k/313M [00:00<7:15:28, 12.0kB/s]
model_final_997cc7.pkl:   0%|          | 24.6k/313M [00:00<2:17:25, 38.0kB/s]
model_final_997cc7.pkl:   0%|          | 57.3k/313M [00:00<55:27, 94.1kB/s]  
model_final_997cc7.pkl:   0%|          | 123k/313M [00:00<24:20, 214kB/s]  
model_final_997cc7.pkl:   0%|          | 180k/313M [00:01<17:38, 296kB/s]
model_final_997cc7.pkl:   0%|          | 246k/313M [00:01<13:42, 380kB/s]
model_final_997cc7.pkl:   0%|          | 319k/313M [00:01<11:13, 464kB/s]
model_final_997cc7.pkl:   0%|          | 369k/313M [00:01<11:07, 468kB/s]
model_final_997cc7.pkl:   0%|          | 442k/313M [00:01<09:44, 535kB/s]
model_final_997cc7.pkl:   0%|          | 508k/313M [00:01<09:18, 560kB/s]
model_final_997cc7.pkl:   0%|          | 557k/313M [00:01<09:48, 531kB/s]
model_final_997cc7.pkl:   0%|          | 631k/313M [00:01<08:56, 582kB/s]
model_final_997cc7.pkl:   0%|          | 819k/313M [0

Video /content/drive/MyDrive/22_FYP42 - Zero-shot Explainable HAR/Datasets/knn_handpicked/5_sample/watching_TV/Copy of Copy of a031-1578C.mp4 processed in 1912.544s shape (177, 17, 2)
Video /content/drive/MyDrive/22_FYP42 - Zero-shot Explainable HAR/Datasets/knn_handpicked/5_sample/watching_TV/Copy of Copy of a031-1601C.mp4 processed in 1938.210s shape (180, 17, 2)


In [ ]:
file__ = np.load("/content/drive/MyDrive/22_FYP42 - Zero-shot Explainable HAR/Datasets/skeleton_npz_outputs/detectron2/knn_handpicked/5_sample/watching_TV/Copy of Copy of a031-1578C.mp4.npz")

In [ ]:
file__.keys()